# Анализ базы данных сервиса для чтения книг по подписке с помощью SQL

# Описание проекта

### Контекст

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша задача как аналитика — проанализировать базу данных. В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

### Цель

Проанализировать базу данных. В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. На их основе сформулировать ценностное предложение для нового продукта.

### Задачи

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Описание исходных данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### План работы

1. Опишем цели исследования;
2. Изучение данных — выведем первые строки и исследуем таблицы;
3. Сделаем по одному SQL-запросу для решения каждого задания:
    - Посчитаем, сколько книг вышло после 1 января 2000 года;
    - Для каждой книги посчитаем количество обзоров и среднюю оценку;
    - Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключим из анализа брошюры;
    - Определим автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками;
    - Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
4. Выведем результат каждого запроса в тетрадке;
5. Опишем выводы по каждой из решённых задач.

## Подключение к базе данных и изучение данных

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Таблица `books` с данными о книгах

In [2]:
query1 = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query1, con = engine)
display(books.info())
books.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица `authors` с данными об авторах

In [3]:
query2 = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query2, con = engine)
display(authors.info())
authors.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица `publishers` с данными об издательствах

In [4]:
query3 = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query3, con = engine)
display(publishers.info())
publishers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица `ratings` с данными о пользовательских оценках книг

In [5]:
query4 = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query4, con = engine)
display(ratings.info())
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица `reviews` с данными о пользовательских обзорах на книги

In [6]:
query5 = '''
            SELECT * 
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query5, con = engine)
display(reviews.info())
reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- имеется к анализу 1000 книг от 636 авторов;
- книги выпущены в 340 издательствах;
- для книг имеются пользовательские оценки (6456 оценок) и обзоры (2793 обзоров).

## Ответы на вопросы

In [7]:
# напишем функцию для выполнения запросов 

def display(query):
    return pd.io.sql.read_sql(query, con = engine)

### Количество книг вышедшие после 1 января 2000 года

In [8]:
books_count = '''
SELECT COUNT(*) AS books_count
FROM books
WHERE publication_date > '2000-01-01';
'''
display(books_count)

,books_count
0,819


После 1 января 2000 года было выпущено **819 книг**.

### Количество обзоров и средняя оценка для каждой книги

In [9]:
ratings_reviews = '''
SELECT b.title AS title,
       b.book_id AS id,
       COUNT(DISTINCT reviews.text) AS count_reviews,
       AVG(ratings.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews ON reviews.book_id = b.book_id
LEFT JOIN ratings ON ratings.book_id = b.book_id
GROUP BY b.title,
         b.book_id
ORDER BY count_reviews DESC
'''
display(ratings_reviews)

,title,id,count_reviews,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


- всего в каталоге **1000 книг**;
- количество обзоров каждой книги в диапазоне **от 0 до 7** (книга Twilight #1 - единственная с 7 обзорами);
- средний рейтинг каждой книги вариьируется **от 1.5 до 5** и не коррелирует с количеством обзоров.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
book_publisher = '''
SELECT p.publisher AS name_publisher,
       COUNT(books.book_id) AS count_of_published_books
FROM publishers AS p
INNER JOIN books ON books.publisher_id = p.publisher_id
WHERE books.num_pages > 50
GROUP BY name_publisher
ORDER BY count_of_published_books DESC
LIMIT 1
'''
display(book_publisher)

,name_publisher,count_of_published_books
0,Penguin Books,42


Издательство **Penguin Books** выпустило **42 книги** (больше всех) объемом более **50 страниц**.

### Автор с самой высокой средней оценкой книг (книги с 50 и более оценками)

In [11]:
best_author = '''
SELECT a.author, AVG(ratings)
FROM (
      SELECT book_id, AVG(rating) ratings
      FROM ratings
      GROUP BY book_id
      HAVING COUNT(*) > 50
      ) rat
JOIN (SELECT book_id, author_id FROM books) b ON rat.book_id=b.book_id
JOIN (SELECT author_id, author FROM authors) a ON b.author_id=a.author_id
GROUP BY a.author
ORDER BY avg(ratings) DESC
LIMIT 1
'''
display(best_author)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самый популярный автор со средним рейтингом в **4.28 баллов из 5** - это **Джоан Роулинг** (автор серии книг о Гарри Поттере) и иллюстратор ее книг Мари Грандпре.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
avg_reviews = '''
SELECT AVG(count) AS avg_review    
FROM (
      SELECT COUNT(text)
      FROM reviews
      FULL OUTER JOIN (SELECT username, 
                              COUNT(rating_id) as cnt 
                       FROM ratings 
                       GROUP BY username) AS count_rating
      ON count_rating.username = reviews.username where cnt > 50
      GROUP BY reviews.username) AS count;                                  
'''
display(avg_reviews)

,avg_review
0,24.333333


В среднем **24,33 обзора** от пользователей, которые поставили более **50 оценок**.

## Выводы 

**Анализ базы данных сервиса для чтения книг по подписке с помощью SQL**

Для анализа базы данных представлено 5 таблиц:

- `books` - содержит 1000 строк и 6 столбцов, содержит данные о книгах;
- `authors` - содержит 636 строк и 2 столбца, содержит данные об авторах;
- `publishers` - содержит 340 строк, 2 столбца, содержит данные об издательствах;
- `rating` - содержит 6456 строк, 4 столбца, содержит данные о пользовательских обзорах;
- `reviews` - содержит 2793 строки, 4 столбца, содержит данные о пользовательских обзорах.

В ходе анализа базы данных сервиса для чтения книг по подписке были даны ответы на 5 вопросов:

*1. Cколько книг вышло после 1 января 2000 года.*
        
        
- после 1 января 2000 года было выпущено **819 книг**.


*2. Для каждой книги посчитать количество обзоров и среднюю оценку.*


- всего в каталоге **1000 книг**;
- количество обзоров каждой книги в диапазоне **от 0 до 7** (книга Twilight #1 - единственная с 7 обзорами);
- средний рейтинг каждой книги вариьируется **от 1.5 до 5** и не коррелирует с количеством обзоров.


*3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (для исключения брошюр).*


- издательство **Penguin Books** выпустило **42 книги** (больше всех) объемом более **50 страниц**.


*4. Определить автора с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками).*


- самый популярный автор со средним рейтингом в **4.28 баллов из 5** - это **Джоан Роулинг** (автор серии книг о Гарри Поттере) и иллюстратор ее книг Мари Грандпре.


*5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.*

- в среднем **24,33 обзора** от пользователей, которые поставили более **50 оценок**.